# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź liczbę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

0. Podłączenie do bazy danych

In [1]:
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

1. Znajdź liczbę wszystkich filmów o tej samej długości.

In [2]:
df = pd.read_sql('SELECT length, COUNT(*) FROM film GROUP BY length ORDER BY length', con=connection)
df

,length,count
0,46,5
1,47,7
2,48,11
3,49,5
4,50,9
...,...,...
135,181,10
136,182,6
137,183,5
138,184,8


2. Znajdź wszystkich klientów mieszkających w tym samym mieście.

In [3]:
df = pd.read_sql('  SELECT city, COUNT(*) \
                    FROM customer \
                    INNER JOIN address \
                        ON customer.address_id = address.address_id \
                    INNER JOIN city \
                        ON address.city_id = city.city_id \
                    GROUP BY city ORDER BY city', con=connection)
df

,city,count
0,A Corua (La Corua),1
1,Abha,1
2,Abu Dhabi,1
3,Acua,1
4,Adana,1
...,...,...
592,Zaria,1
593,Zeleznogorsk,1
594,Zhezqazghan,1
595,Zhoushan,1


3. Oblicz średni koszt wypożyczenia wszystkich filmów.

In [4]:
df = pd.read_sql('  SELECT AVG(amount) \
                    FROM payment', con=connection)
df

,avg
0,4.200606


4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.

In [5]:
df = pd.read_sql('  SELECT category.name, COUNT(*) \
                    FROM film \
                    INNER JOIN film_category \
                        ON film_category.film_id = film.film_id \
                    INNER JOIN category \
                        ON category.category_id = film_category.category_id \
                    GROUP BY category.name \
                    ORDER BY name', con=connection)
df

,name,count
0,Action,64
1,Animation,66
2,Children,60
3,Classics,57
4,Comedy,58
5,Documentary,68
6,Drama,62
7,Family,69
8,Foreign,73
9,Games,61


5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.

In [6]:
df = pd.read_sql('  SELECT country.country, COUNT(*) \
                    FROM customer \
                    INNER JOIN address \
                        ON customer.address_id = address.address_id \
                    INNER JOIN city \
                        ON address.city_id = city.city_id \
                    INNER JOIN country \
                        ON country.country_id = city.country_id \
                    GROUP BY country.country ORDER BY country.country', con=connection)
df

,country,count
0,Afghanistan,1
1,Algeria,3
2,American Samoa,1
3,Angola,2
4,Anguilla,1
...,...,...
103,Vietnam,6
104,"Virgin Islands, U.S.",1
105,Yemen,4
106,Yugoslavia,2


6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.

In [7]:
df = pd.read_sql('  SELECT store.store_id, COUNT(*) as clients_num\
                    FROM customer \
                    INNER JOIN store \
                        ON store.store_id = customer.store_id \
                    GROUP BY store.store_id \
                    HAVING COUNT(*) BETWEEN 100 AND 300', con=connection)
df

,store_id,clients_num
0,2,273


7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.

In [8]:
# zakładam, że klient obejrzał film który wypożyczył
df = pd.read_sql('  SELECT customer.customer_id, customer.first_name, customer.last_name, SUM(film.length) as watch_time \
                    FROM customer \
                    INNER JOIN rental \
                        ON rental.customer_id = customer.customer_id \
                    INNER JOIN inventory \
                        ON inventory.inventory_id = rental.inventory_id \
                    INNER JOIN film \
                        ON film.film_id = inventory.film_id \
                    GROUP BY customer.customer_id \
                    HAVING SUM(film.length) > 200', con=connection)
df

,customer_id,first_name,last_name,watch_time
0,251,Vickie,Brewer,3484
1,106,Connie,Wallace,2795
2,120,Sylvia,Ortiz,3567
3,285,Miriam,Mckinney,2672
4,264,Gwendolyn,May,2890
...,...,...,...,...
594,328,Jeffrey,Spear,2678
595,263,Hilda,Hopkins,3080
596,144,Clara,Shaw,4808
597,7,Maria,Miller,3494


8. Oblicz średnią wartość wypożyczenia filmu.

In [9]:
# ODP. jak w 3 zadaniu

9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.

In [10]:
df = pd.read_sql('  SELECT category.name, AVG(film.length) as avg_time\
                    FROM film \
                    INNER JOIN film_category \
                        ON film_category.film_id = film.film_id \
                    INNER JOIN category \
                        ON category.category_id = film_category.category_id \
                    GROUP BY category.name \
                    ORDER BY category.name', con=connection)
df 

,name,avg_time
0,Action,111.609375
1,Animation,111.015152
2,Children,109.800000
3,Classics,111.666667
4,Comedy,115.827586
5,Documentary,108.750000
6,Drama,120.838710
7,Family,114.782609
8,Foreign,121.698630
9,Games,127.836066


10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.

In [11]:
df = pd.read_sql('  SELECT category.name, MAX(film.length) as longest_time\
                    FROM film \
                    INNER JOIN film_category \
                        ON film_category.film_id = film.film_id \
                    INNER JOIN category \
                        ON category.category_id = film_category.category_id \
                    GROUP BY category.name \
                    ORDER BY category.name', con=connection)
df 

,name,longest_time
0,Action,185
1,Animation,185
2,Children,178
3,Classics,184
4,Comedy,185
5,Documentary,183
6,Drama,181
7,Family,184
8,Foreign,184
9,Games,185


In [12]:
df = pd.read_sql('SELECT category_id FROM category', con=connection)
df 

,category_id
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [13]:
import main as m

In [21]:
category_id = 5
m.film_in_category(category_id)

,title,languge,name
0,Airplane Sierra,English,Action
1,Anthem Luke,English,Action
2,Bringing Hysterical,English,Action
3,Caper Motions,English,Action
4,Cat Coneheads,English,Action
...,...,...,...
923,Valley Packer,English,Travel
924,Velvet Terminator,English,Travel
925,Vertigo Northwest,English,Travel
926,Wisdom Worker,English,Travel


In [15]:
category_id = 5
m.number_films_in_category(category_id)

,category,count
0,Comedy,58


In [16]:
min_length = 46
max_length = 46
m.number_film_by_length(min_length, min_length)

,length,count
0,46,5


In [17]:
city = 'Athenai'
m.client_from_city(city)

,city,first_name,last_name
0,Athenai,Linda,Williams


In [18]:
length = 48
m.avg_amount_by_length(length)

,length,avg
0,48,4.295389


In [26]:
sum_min = 1000
m.client_by_sum_length(sum_min)

,first_name,last_name,sum
0,Brian,Wyman,1265
1,Antonio,Meek,1451
2,Leona,Obrien,1588
3,Katherine,Rivera,1615
4,Tiffany,Jordan,1667
...,...,...,...
594,Clara,Shaw,4808
595,Wesley,Bull,4808
596,Tammy,Sanders,5065
597,Eleanor,Hunt,5360


In [20]:
name = 'Drama'
m.category_statistic_length(name)

,category,avg,sum,min,max
0,Drama,120.83871,7492,46,181
